Prob.py

In [ ]:
from abc import ABC, abstractmethod

class Problem(ABC):
  def __init__(self) -> None:
    self.init_state = []
    self.n = 0
  
  @abstractmethod
  def goal_test(self, state) -> bool: pass # Check if state is final

  @abstractmethod
  def actions(self, state): pass # Show available actions 

  @abstractmethod
  def g(self, from_state, action): pass

  @abstractmethod
  def h(self, state): pass #Heuristic value  

  @abstractmethod
  def random_state(self): pass # Return random state of the problem

nQueens.py

In [ ]:
from Prob import* 
import random

class nQueens(Problem):
    def __init__(self, n) -> None: 
        self.init_state = tuple(random.randrange(0, n) for _ in range (n))
        self.n = n
    
    def actions(self, state) -> bool:
        action = []
        for row in range(self.n):
            for col in range(self.n):
                if col != state[row]:
                    new_action = list(state[:])
                    new_action[row] = col
                    action.append(tuple(new_action))
        return action
    
    def result(self, state, action): 
        newState = list(state[:])
        newState[action[0]] = action[1] # Location of a queen on a column 
        return tuple(newState)
    
    def conflict_check(self, r1, c1, r2, c2): #Check vertically, horizontally and diagonally
        return r1 == r2 or c1 == c2 or abs(r1 - r2) == abs(c1 - c2)

    def goal_test(self, state: 'tuple[int]') -> bool:
        if self.h(state):
            return False
        return True
        
    def g(self, from_state, to_state): 
        return self.h(to_state)    
        
    def h(self, state: 'tuple[int]'):
        conflict_count = 0
        for i in range(len(state) - 1):
            for j in range(i + 1, len(state)):
                if self.conflict_check(i, state[i], j, state[j]): # If conflict occurs, add 1 to the current conflict value
                    conflict_count += 1
        return conflict_count

    def random_state(self) -> 'tuple[int]':
        return tuple([random.randrange(0, self.n) for _ in range(self.n)])



A*

In [ ]:
from nQueens import *
import heapq

class A_Star:
    class Node: 
        def __init__(self, problem: Problem, state,  parent: 'A_Star.Node' = None, path_cost = 0 ) -> None:
            self.state = state
            self.problem = problem
            self.h = problem.h(state)
            self.g = path_cost + self.h
            self.parent = parent
            

        def child_node(self, action): # Return child node of given action 
            return A_Star.Node(self.problem, action, self, self.h)

        def __repr__(self): 
            return "<Node {}( g = {}, h = {})>".format(self.state, self.g, self.h)

        def __lt__(self, other: 'A_Star.Node'):
            return (self.g + self.h) < (other.g + other.h)

        def path(self) -> list:  # Returns list of nodes from the current node to root node
            node, path_back = self, [] 
            while node:
                path_back.append(node)
                node = node.parent
            path_back.reverse()
            return path_back

        def print_board(self):
            k = len(self.state)
            for i in range(k):
                for j in range(k):
                    print("Q " if i == self.state[j] else "* " , end='')
                print()
                   

    def __init__(self, problem: Problem) -> None:
        self.problem = problem
        self.root = A_Star.Node(problem, problem.init_state) 
    
    def solution(self): # Returns actions of nodes from the current node to root node
            frontier = [self.root]
            heapq.heapify(frontier)
            explored = set()
            explored.add(self.root.state)
            while frontier:
                cur = heapq.heappop(frontier)
                if not cur.h:
                    return cur
                
                for action in self.problem.actions(cur.state):
                    new_node = cur.child_node(action)
                    if new_node.state not in explored:
                        heapq.heappush(frontier, new_node)
                        explored.add(new_node.state)
            return None
    


UCS

In [ ]:
from nQueens import *
import heapq

class UCS:
    class Node: 
        def __init__(self, problem: Problem, state, parent: 'UCS.Node' = None, path_cost=0) -> None:
            self.state = state
            self.problem = problem
            self.g = path_cost
            self.h = 0
            self.parent = parent
            
        def child_node(self, action): # Return child node of given action 
            return UCS.Node(self.problem,action, self, self.g + self.problem.g(self.state, action))

        def __repr__(self): 
            return "<Node {}( g = {})>".format(self.state, self.g)

        def __lt__(self, other: 'UCS.Node'):
            return self.g < other.g
        
        def path(self) -> list:  # Returns list of nodes from the current node to root node
            node, path_back = self, [] 
            while node:
                path_back.append(node)
                node = node.parent
            path_back.reverse()
            return path_back

        def print_board(self):
            k = len(self.state)
            for i in range(k):
                for j in range(k):
                    print("Q " if i == self.state[j] else "* " , end='')
                print()
                   

    def __init__(self, problem: Problem) -> None:
        self.problem = problem
        self.root = UCS.Node(problem, problem.init_state) 
    
    def solution(self): # Returns actions of nodes from the current node to root node
        frontier = [self.root]
        heapq.heapify(frontier)
        explored = set()
        while frontier:
            cur = heapq.heappop(frontier)
            if self.problem.goal_test(cur.state):
                return cur
            if cur.state in explored: continue

            explored.add(cur.state)
            for action in self.problem.actions(cur.state):
                newNode = cur.child_node(action)
                if newNode.state not in explored:
                    heapq.heappush(frontier, newNode)
        return None

Genetic Algorithm

In [ ]:
from nQueens import* 
import random

class Genetic:
  POPULATION_SIZE = 250
  MAX_GEN = 100
  MUTATION_PROB = 0.1

  def __init__(self, problem: Problem) -> None:
    self.problem = problem
    self.n = problem.n
    self.max_fitness = (self.n * (self.n - 1)) // 2
    self.population = [problem.random_state() for _ in range(Genetic.POPULATION_SIZE)]
    self.gen = 0
    self.solution = None
  
  def fitness(self, chromosome):
    return self.max_fitness - self.problem.h(chromosome)

  def calc_prob(self, fitness): #Calculate probabilty with current fitness
    return fitness / self.max_fitness

  def cross_over(self, chrom_1, chrom_2):
    n = len(chrom_1)
    split = random.randrange(0, n)
    child = chrom_1[:split] + chrom_2[split:]
    return child

  def mutate(self, chrom):
    pos = random.randrange(0, len(chrom))
    new_chrom = list(chrom)
    new_chrom[pos] = random.randrange(0, self.n)
    return tuple(new_chrom)

  def print_board(self):
        print("Population size: {}".format(Genetic.POPULATION_SIZE))
        print("Mutation probability: {}.".format(Genetic.MUTATION_PROB))
        print("Solved in generation: {}".format(self.gen))
        k = len(self.solution)
        for i in range(k):
            for j in range(k):
                print("Q " if i == self.state[j] else "* " , end='')
            print()  

  def solution(self, no_limit=False):
    while no_limit or self.gen != Genetic.MAX_GEN:
      all_fitness = [self.fitness(chrom) for chrom in self.population]
      if self.max_fitness in all_fitness:
        self.solution = self.population[all_fitness.index(self.max_fitness)]
        return

      # Calculate probabilitiy for each chromosome to be choosen for next generation
      self.probs = [self.calc_prob(f) for f in all_fitness]
      new_population = []
      for _ in range(len(self.population)):
        # Selection
        first, second = random.choices(self.population, weights=self.probs, k=2)
        # Cross-over
        child = self.cross_over(first, second)

        # Mutation
        if random.random() < Genetic.MUTATION_PROB:
          child = self.mutate(child)

        new_population.append(child)

      del self.population
      self.population = new_population
      self.gen += 1


main func

In [ ]:
from time import perf_counter_ns
import tracemalloc
from Prob import *
from nQueens import *
from A_Star import *
from UCS import *
from Genetic import* 

N = 8
problem = nQueens(N)
testcase = 3
def queen_test(func):
    def calculate():
        t = []
        m = []
        for i in range(testcase):
            tracemalloc.start()
            t1 = perf_counter_ns()
            print("Testcase no.{} :".format(i+1))
            func()
            t2 = perf_counter_ns()
            peak = tracemalloc.get_traced_memory()[1]
            tracemalloc.stop()
            t.append((t2 - t1) / 10**6)
            m.append(peak / 1024**2)
            print("Runtime of testcase no.{} : {runtime:.2f} ms".format(i+1, runtime = t[i]))
            print("Memory of testcase no.{} : {mem:.4f} MB".format(i+1, mem = m[i]))

        print()
        print("Average time: {avg_runtime:.2f} ms".format(avg_runtime = sum(t) / testcase))
        print("Average memory usage: {avg_mem:.4f} MB.".format(avg_mem = sum(m) / testcase))
    return calculate

@queen_test
def run_astar_search():
    astar = A_Star(nQueens(N))
    astar.solution().print_board()

@queen_test
def run_ucs_search():
    ucs = UCS(nQueens(N))
    ucs.solution().print_board()

@queen_test
def run_genetic():
    gene = Genetic(nQueens(N))
    gene.solution(1)
    gene.print_board()

def main():
    print("1. A* Search")
    print("2. UCS Search")
    print("3. Genetic Algorithm")
    enter = int(input("Choose an algorithm to solve N-queens: "))
    if enter == 1: 
        run_astar_search()
    elif enter == 2:
        run_ucs_search()
    elif enter == 3:
        run_genetic()

main()

ModuleNotFoundError: ignored